# 07_03: More data formats with Pandas

In [3]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In addition to simple ascii tables, pandas supports many other formats. In some cases you'll need to install other packages to support that functionality. Here's a table!

* JSON (used in web applications);
* HTML and XML (which we may scrape directly from a website);
* Microsoft Excel sheets;
* HDF, the hierarchical format for scientific data;
* the very efficient binary formats from the Apache (uh-PATCH-ee) software foundation, such as feather and parquet (par-KAY);
* proprietary statistics-software formats such as SAS (sass), Stata (stay-tah), and SPSS;
* SQL databases;
* last, the internal binary Python format pickle.

This list is not exhaustive; other formats are supported by third-party packages, so it's always worth googling or asking chatgpt.

In this video we'll concentrate on a few useful formats, but our considerations will apply more generally.

In [4]:
df = pd.read_csv('Planets.csv', thousands=',', parse_dates=['FirstVisited'],
                 dtype={'Diameter': np.float64, 'MeanTemperature': np.float64,
                        'Rings': 'category', 'MagneticField': 'category'})
df.head()

,Planet,Mass,Diameter,DayLength,SunDistance,OrbitPeriod,OrbitVelocity,MeanTemperature,SurfacePressure,Moons,Rings,MagneticField,FirstVisited,FirstMission
0,MERCURY,0.330,4879.0,4222.6,57.9,88.0,47.4,167.0,0.00,0,No,Yes,1974-03-29,Mariner 10
1,VENUS,4.870,12104.0,2802.0,108.2,224.7,35.0,464.0,92.00,0,No,No,1962-08-27,Mariner 2
2,EARTH,5.970,12756.0,24.0,149.6,365.2,29.8,15.0,1.00,1,No,Yes,NaT,NaN
3,MOON,0.073,3475.0,708.7,NaN,27.3,1.0,-20.0,0.00,0,No,No,1959-09-12,Luna 2
4,MARS,0.642,6792.0,24.7,227.9,687.0,24.1,-65.0,0.01,2,No,No,1965-07-15,Mariner 4


After we've done the careful work of import text data, we can save our DataFrame in a binary format that will preserve the dtypes and other details. The simplest way to do so is with Python's serializing capability (known as _pickling_).

Beware that pickles may not be compatible across different versions of Python and pandas, so the format is best used internally within an application, rather than as to exchange data.

So here's writing to pickle...

In [5]:
df.to_pickle('planets.pkl')

...and here's reading; nothing to it.

In [6]:
pd.read_pickle('planets.pkl').head()

,Planet,Mass,Diameter,DayLength,SunDistance,OrbitPeriod,OrbitVelocity,MeanTemperature,SurfacePressure,Moons,Rings,MagneticField,FirstVisited,FirstMission
0,MERCURY,0.330,4879.0,4222.6,57.9,88.0,47.4,167.0,0.00,0,No,Yes,1974-03-29,Mariner 10
1,VENUS,4.870,12104.0,2802.0,108.2,224.7,35.0,464.0,92.00,0,No,No,1962-08-27,Mariner 2
2,EARTH,5.970,12756.0,24.0,149.6,365.2,29.8,15.0,1.00,1,No,Yes,NaT,NaN
3,MOON,0.073,3475.0,708.7,NaN,27.3,1.0,-20.0,0.00,0,No,No,1959-09-12,Luna 2
4,MARS,0.642,6792.0,24.7,227.9,687.0,24.1,-65.0,0.01,2,No,No,1965-07-15,Mariner 4


Personally, I like the very efficient and portable formats developed by the Apache arrow project, feather and parquet. For large data files these can be a few times faster than pickle, and many times faster than text. On the other hand, you'll need pickle if your table contains custom Python objects.

In [7]:
df.to_feather('planets.feather')

In [8]:
pd.read_feather('planets.feather').head()

,Planet,Mass,Diameter,DayLength,SunDistance,OrbitPeriod,OrbitVelocity,MeanTemperature,SurfacePressure,Moons,Rings,MagneticField,FirstVisited,FirstMission
0,MERCURY,0.330,4879.0,4222.6,57.9,88.0,47.4,167.0,0.00,0,No,Yes,1974-03-29,Mariner 10
1,VENUS,4.870,12104.0,2802.0,108.2,224.7,35.0,464.0,92.00,0,No,No,1962-08-27,Mariner 2
2,EARTH,5.970,12756.0,24.0,149.6,365.2,29.8,15.0,1.00,1,No,Yes,NaT,None
3,MOON,0.073,3475.0,708.7,NaN,27.3,1.0,-20.0,0.00,0,No,No,1959-09-12,Luna 2
4,MARS,0.642,6792.0,24.7,227.9,687.0,24.1,-65.0,0.01,2,No,No,1965-07-15,Mariner 4


In [301]:
# also: df.to_parquet('planets.parquet')
#       pd.read_parquet('planets.parquet')

Let's move on to JSON, the native JavaScript data format. This is strictly speaking a text format, but it is meant more for machine reading and writing than for people. In addition, the JSON syntax is very close to Python lists and dicts. The main difference is that in JavaScript and JSON dictionary keys do not need to be quoted.

We'll use a simplified version of the planets dataframe to exemplify JSON reading. The important point here is that there are different ways to organize a DataFrame using dicts and lists.

Perhaps the most straightforward way is having one dict for each record. This is also what we would need, for instance, in the JavaScript plotting library `d3`.

**Planets-records.json**

    [ { "Planet": "MERCURY",
        "Mass": 0.33,
        "FirstMission": "Mariner 10" },
      { "Planet": "VENUS",
        "Mass": 4.87,
        "FirstMission": "Mariner 2" },
      { "Planet": "EARTH",
        "Mass": 5.97,
        "FirstMission": null }, ...]

pandas has no problems loading this

In [41]:
pd.read_json('Planets-records.json').head()

,Planet,Mass,FirstMission
0,MERCURY,0.330,Mariner 10
1,VENUS,4.870,Mariner 2
2,EARTH,5.970,None
3,MOON,0.073,Luna 2
4,MARS,0.642,Mariner 4


To write in this variant of JSON, we'd use `to_json` with the option `orient='records'`.

In [44]:
# df.to_json('Planets-records.json', orient='records')

If you have an explicit index, you'd probably use a dictionary of dictionaries

**Planets-index.json**
    
    { "MERCURY": { "Mass": 0.33,
                    "FirstMission": "Mariner 10" },
      "VENUS":   { "Mass": 4.87,
                    "FirstMission": "Mariner 2" },
      "EARTH":   { "Mass": 5.97,
                   "FirstMission": null }, ... }

But you'd need to tell pandas. Otherwise you get the _transpose_ of this dataframe.

In [76]:
pd.read_json('Planets-index.json', orient='index')

,Mass,FirstMission
MERCURY,0.3300,Mariner 10
VENUS,4.8700,Mariner 2
EARTH,5.9700,None
MOON,0.0730,Luna 2
MARS,0.6420,Mariner 4
JUPITER,1898.0000,Pioneer 10
SATURN,568.0000,Pioneer 11
URANUS,86.8000,Voyager 2
NEPTUNE,102.0000,Voyager 2
PLUTO,0.0146,New Horizons


In [69]:
pd.read_json('Planets-index.json')

,MERCURY,VENUS,EARTH,MOON,MARS,JUPITER,SATURN,URANUS,NEPTUNE,PLUTO
Mass,0.33,4.87,5.97,0.073,0.642,1898.0,568.0,86.8,102.0,0.0146
FirstMission,Mariner 10,Mariner 2,NaN,Luna 2,Mariner 4,Pioneer 10,Pioneer 11,Voyager 2,Voyager 2,New Horizons


To save space, you'd instead separate the labels (both index and column), and condense the values in a list of lists. This is known as a `split` orientation.

**Planets-split.json**

    { "columns": ["Planet", "Mass", "FirstMission"],
      "index":   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
      "data":    [ [ "MERCURY", 0.33, "Mariner 10" ],
                   [ "VENUS",   4.87, "Mariner 2" ],
                   [ "EARTH",   5.97, null ], ... ] }

In [74]:
pd.read_json('Planets-split.json', orient='split')

,Planet,Mass,FirstMission
0,MERCURY,0.3300,Mariner 10
1,VENUS,4.8700,Mariner 2
2,EARTH,5.9700,None
3,MOON,0.0730,Luna 2
4,MARS,0.6420,Mariner 4
5,JUPITER,1898.0000,Pioneer 10
6,SATURN,568.0000,Pioneer 11
7,URANUS,86.8000,Voyager 2
8,NEPTUNE,102.0000,Voyager 2
9,PLUTO,0.0146,New Horizons


Other `read_json` options are similar to `read_csv` (for instance, they determine date and float parsing, or column dtypes). Missing values are always rendered as `null` in JSON, which is mapped back to the correct missing-data value in pandas.

(Code to make these...)

    print(json.dumps(json.loads(df.set_index('Planet')[['Mass','FirstMission']].to_json(orient='index')), indent=True))
    json.dump(json.loads(df.set_index('Planet')[['Mass','FirstMission']].to_json(orient='index')), open('Planets-index.json','w'))

    print(json.dumps(json.loads(df[['Planet','Mass','FirstMission']].to_json(orient='records')), indent=True))
    json.dump(json.loads(df[['Planet','Mass','FirstMission']].to_json(orient='records')), open('Planets-records.json','w'))

    print(json.dumps(json.loads(df[['Planet','Mass','FirstMission']].to_json(orient='split')), indent=True))
    json.dump(json.loads(df[['Planet','Mass','FirstMission']].to_json(orient='split')), open('Planets-split.json','w'))

Last, I want to show you how pandas can parse HTML. We don't even need the file: we can just provide a URL---for instance a table of Athletics medals at the 2024 olympics, from wikipedia. (Navigate to https://en.wikipedia.org/wiki/Athletics_at_the_2024_Summer_Olympics).

For convenience, I have included the file in this repository.

In [121]:
# pd.read_html('https://en.wikipedia.org/wiki/Athletics_at_the_2024_Summer_Olympics')

The reader tries to parse all tables in the file, so we get back a list...

But we can use `match` to look for a specific word and thus select a subset of tables. We could also use an HTML attribute such as id, but that doesn't help us with these wikipedia tables. We get a table of medals by country.

In [178]:
# select on attribute with attrs = {'id': ...}
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank')[0]

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,United States,14,11,9,34
1,2,Kenya,4,2,5,11
2,3,Canada,3,1,1,5
3,4,Netherlands,2,1,3,6
4,5,Spain,2,1,1,4
5,6,Norway,2,1,0,3
6,7,Great Britain,1,4,5,10
7,8,Jamaica,1,3,2,6
8,9,Ethiopia,1,3,0,4
9,10,Australia,1,2,4,7


Parsing options are similar to `read_csv`. One thing we can do here is to use the country as the index.

In [157]:
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank', index_col=1)[0].tail()

,Rank,Gold,Silver,Bronze,Total
NOC,,,,,
Poland,37,0,0,1,1
Puerto Rico,37,0,0,1,1
Qatar,37,0,0,1,1
Zambia,37,0,0,1,1
Totals (43 entries),Totals (43 entries),48,48,49,145


We should also remove the last row. We can take our own totals!

In [167]:
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank', index_col=1, skiprows=[44])[0].tail()

,Rank,Gold,Silver,Bronze,Total
NOC,,,,,
Czech Republic,37,0,0,1,1
Poland,37,0,0,1,1
Puerto Rico,37,0,0,1,1
Qatar,37,0,0,1,1
Zambia,37,0,0,1,1


You can imagine how useful it can be to grab data on the internet and get it into pandas for analysis. However, as we will see in the next video, HTML data often needs significant cleaning and reorganization.